In [ ]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json


# Call Earthquake API

In [ ]:
date = "2023-10-01"
url = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

date = datetime.strptime(date, "%Y-%m-%d")
yesterday = date - relativedelta(days=1)

query_params = {
    'starttime': yesterday.replace(day=1),
    'endtime': yesterday,
    'format': 'geojson',
}

try:
    response = requests.get(url, params=query_params)

    if response.status_code == 200:
        response_json = response.json()
    else:
        print(f"Request failed with status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")


# Write and Read data

In [ ]:
with open("geojson_data.json", "w", encoding="utf-8") as f:
    json.dump(response_json, f)


In [ ]:
# response_json
with open("geojson_data.json", "r", encoding="utf-8") as f:
    geojson_json = json.load(f)

geojson_json


In [ ]:
df = pd.json_normalize(response_json["features"])
df.columns
df.head()


In [ ]:
with open("geojson_data.json", "r") as f:
    df = pd.json_normalize(json.load(f)["features"])
renaming = {'properties.mag': 'properties_magnitude',
'properties.place':  'properties_place',
'properties.time':  'properties_time',
'properties.updated':  'properties_updated',
'properties.felt':  'properties_felt_count',
'properties.alert':  'properties_alert',
'properties.status':  'properties_status',
'properties.tsunami':  'properties_tsunami',
'properties.sig':  'properties_significance',
'properties.nst':  'properties_seismic_station_count',
'properties.type':  'properties_type',
'properties.title':  'properties_title'}
df.rename(columns=renaming, inplace=True)
df.columns
def split_coordinates(row: pd.Series):
    coordinates = row['geometry.coordinates']
    row['longitude'] = coordinates[0]
    row['latitude'] = coordinates[1]
    row['elevation'] = coordinates[2]
    return row

df = df.apply(split_coordinates, axis='columns')


In [ ]:
".".join("data/toto.geojson_data.json".split(".")[:-1]) + ".parquet"


In [ ]:
import pandas as pd

with open("geojson_data.json", "r") as f:
    df = pd.json_normalize(json.load(f)["features"])

df = pd.read_parquet("usgs_data-geojson_data_2022-12-31.parquet")
df.head()
df.info()


In [ ]:
len(df[['type', 'id', 'properties_magnitude', 'properties_place',
            'properties_time', 'properties_updated',
            'properties_felt_count', 'properties_alert',
            'properties_status', 'properties_tsunami',
            'properties_significance', 'properties_seismic_station_count',
            'properties_type', 'properties_title', 'longitude', 'latitude', 'elevation']].columns) == len(df.columns)

set(df[['type', 'id', 'properties_magnitude', 'properties_place',
            'properties_time', 'properties_updated',
            'properties_felt_count', 'properties_alert',
            'properties_status', 'properties_tsunami',
            'properties_significance', 'properties_seismic_station_count',
            'properties_type', 'properties_title', 'longitude', 'latitude', 'elevation']].columns) - set(df.columns)


In [ ]:
import numpy as np

df["properties_felt_count"] = df["properties_felt_count"].replace(np.nan, 0).astype(int)
df["properties_felt_count"]


In [ ]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
df.parallel_apply(split_coordinates, axis='columns')

df.head(2)


In [ ]:
from datetime import datetime
datetime.utcfromtimestamp(1696031889550/1000)


In [ ]:
str(datetime.utcfromtimestamp(1577750367859/1000).replace(microsecond=0))


In [ ]:
df["properties_felt_count"].astype("Int64")


In [ ]:
###
import pandas as pd
import json
with open("usgs_data-geojson_data_2022-11-30-bronze.json", "r") as f:
        json_data = json.load(f)
        df = pd.json_normalize(json_data["features"])

df["properties.time"].head(1)


In [ ]:
# (df["properties_time"]/1000).astype('datetime64[s]')
(df["properties.time"]).astype('datetime64[ms]')
pd.concat([df, pd.DataFrame(df["geometry.coordinates"].tolist(), columns=["geo1", "geo2", "geo3"])], axis=1)


In [ ]:
from datetime import datetime

datetime.utcfromtimestamp(1669765859910/1000)


In [ ]:
df
